### Evaluating the m5 limits ###

Create values to be used in `rubin_sim.utils.sysEngVals`

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import syseng_throughputs as st

In [2]:
pd.set_option('display.precision', 3)

In [3]:
# Read bandpass information.
defaultDirs = st.setDefaultDirs()
addLosses = True
# Use the X=1.0 atmosphere!
atmosphere = st.readAtmosphere(defaultDirs['atmosphere'], atmosFile='atmos_10_aerosol.dat')
hardware, system = st.buildHardwareAndSystem(defaultDirs, addLosses, atmosphereOverride=atmosphere)

/Users/yoachim/git_repos/rubin_sim/rubin_sim/phot_utils/bandpass.py:104: UserWarning: Wavelength sampling of 1.0 nm is > 0.2 nm, this may not work well with a Sed object. Consider resampling with resample_bandpass method.
  warnings.warn(
/Users/yoachim/git_repos/rubin_sim/rubin_sim/phot_utils/bandpass.py:104: UserWarning: Wavelength sampling of 800.0 nm is > 0.2 nm, this may not work well with a Sed object. Consider resampling with resample_bandpass method.
  warnings.warn(
/Users/yoachim/git_repos/rubin_sim/rubin_sim/phot_utils/bandpass.py:104: UserWarning: Wavelength sampling of 0.2 nm is > 0.2 nm, this may not work well with a Sed object. Consider resampling with resample_bandpass method.
  warnings.warn(


In [4]:
# --note these are 1x30s and the overview paper uses 2x15s!!! 
# (this is used as the 'base exposure time' in sysEngVals and the m5 calculation from Cm/dCm_infinity)
# Note also this is using fiducial (old) values of the seeing to set the m5 values as well. 
# thus the resulting values are different than those reported in pstn-054, where the Cm/dCm reported values are 
# for the median delivered seeing in the opsim databases.

m5_std = st.makeM5(hardware, system, exptime=30, nexp=1, readnoise=8.8, othernoise=0, darkcurrent=0.2)

m5_std

,FWHMeff,FWHMgeom,skyMag,skyCounts,Zp_t,Tb,Sb,kAtm,gamma,Cm,dCm_infinity,dCm_double,m5,sourceCounts,m5_fid,m5_min
u,0.92,0.808,22.961,80.845,27.030,0.037,0.058,0.502,0.038,23.390,0.372,0.220,24.074,456.445,23.9,23.4
g,0.87,0.767,22.258,411.095,28.381,0.127,0.154,0.214,0.039,24.507,0.099,0.049,24.900,740.702,25.0,24.6
r,0.83,0.734,21.197,818.694,28.156,0.103,0.116,0.126,0.039,24.490,0.052,0.024,24.403,950.758,24.7,24.3
i,0.80,0.710,20.477,1173.498,27.857,0.078,0.085,0.096,0.039,24.373,0.037,0.016,23.966,1079.948,24.0,23.6
z,0.78,0.693,19.600,1782.977,27.461,0.054,0.058,0.069,0.039,24.203,0.025,0.010,23.385,1280.706,23.3,22.9
y,0.76,0.677,18.610,2371.147,26.680,0.026,0.031,0.170,0.039,23.769,0.019,0.007,22.485,1429.565,22.1,21.7


In [5]:
# Write out values of dictionaries that can be pasted into sims_utils
output_cols = ['Zp_t', 'Tb', 'gamma', 'kAtm', 'Cm', 'dCm_infinity', 'dCm_double', 'skyMag']
for column in output_cols:
    result = '{'
    for filtername, val in zip(m5_std.index, m5_std[column]):
        result += '"%s": %f, ' % (filtername, val)
    result += '}'
    result = result.replace(', }', '}')
    result = 'self.'+column+ ' = ' + result
    print(result)

self.Zp_t = {"u": 27.029564, "g": 28.380921, "r": 28.155692, "i": 27.856986, "z": 27.460868, "y": 26.680272}
self.Tb = {"u": 0.036517, "g": 0.126775, "r": 0.103025, "i": 0.078245, "z": 0.054326, "y": 0.026471}
self.gamma = {"u": 0.037809, "g": 0.038650, "r": 0.038948, "i": 0.039074, "z": 0.039219, "y": 0.039300}
self.kAtm = {"u": 0.502300, "g": 0.213738, "r": 0.125886, "i": 0.096177, "z": 0.068618, "y": 0.169539}
self.Cm = {"u": 23.390279, "g": 24.506790, "r": 24.489907, "i": 24.372557, "z": 24.202751, "y": 23.769170}
self.dCm_infinity = {"u": 0.371925, "g": 0.098515, "r": 0.051968, "i": 0.036845, "z": 0.024581, "y": 0.018608}
self.dCm_double = {"u": 0.220168, "g": 0.049343, "r": 0.024143, "i": 0.016277, "z": 0.010146, "y": 0.007225}
self.skyMag = {"u": 22.960682, "g": 22.257757, "r": 21.196728, "i": 20.477411, "z": 19.599575, "y": 18.610368}
